In [2]:
import PyPDF2
from PyPDF2 import PdfFileReader as r
import pycountry

# File names
file_names = ['FY' + str(num).zfill(2) + '.pdf' for num in range(6, 17) if num is not 9]

# get text
def get_page_text(file_name, pageno=1):
    if (pageno <= 0):
        raise ValueError("Page number can't neither be negative nor zero")
    
    pageno -= 1
    
    pdf = r(open(file_name, 'rb'))
    
    if (pageno > pdf.numPages): raise ValueError("Page index can't be bigger than number of pages")
    
    return pdf.getPage(pageno).extractText()

# Gets all text from a pdf 
def get_all_text(file_name):
    
    # Opens a file
    pdf = r(open(file_name, 'rb'))
    
    # Read all of the texts
    txt = ''.join(
        text for text in 
        [
            pdf.getPage(num).extractText()
            for num in range(pdf.numPages)
        ]
    )
    
    return txt

In [3]:
txt_list = [txt.split('\n') for txt in get_all_text(file_names[0]).split('%\n')]

# Adding first element
refusal_rates = [{
        "country" : pycountry.countries.get(name=txt_list[0][-2]),
        "rate" : float(txt_list[0][-1])
    }]


# Interface for pycountry
def get_country(c):
    return pycountry.countries.get(name=c)

# Gets country name
def country_name(name):
    name = name.replace(
        'St.', 'Saint').replace(
        '-', ' and ').replace(
        'Burma', 'Myanmar').replace(
        'Cape Verde', 'Cabo Verde').replace(
        'British Virgin Islands', 'Virgin Islands, British'
    ).replace('Federated States of Micronesia', 'Micronesia, Federated States of').replace('Hong Kong S. A. R.', 'Hong Kong').replace(
        'East Timor', 'Timor-Leste'
    ).replace('China - Mainland', 'China')
    
    # Remove parenthesis
    if '(' in name or '-' in name:
        name = name.split(' ')[0]
        
        
    try:
        return get_country(name)
    except:
        name = ''.join(c.name for c in list(pycountry.countries) if name in c.name)
        try:
            return get_country(name)
        except:
            return None
    return None

In [4]:
txt_list[47]

["Cote D'Ivoire", '51.3']

In [10]:
# replace - with space-space
# replace korea with - Democratic People's Republic of Korea
# replace British Virgin Islands with Virgin Islands (British)
# replace Hong Kong gibberish with Hong Kong and aggregate the rates
# Add a placeholder for Jerusalem - Capital of Israel
#d Convert laos to Lao
#d Convert Macau to Macao
#d Convert Macedonia to Republic of Macedonia
# Convert North Korea to Democratic People's Republic of Korea
# Convert South Korea to Republic of Korea
#d Convert Zaire to Congo-Kinshasa
#d Convert British Virgin Islands to Virgin Islands (British) 
#d Replace St. with Saint
#d Convert Syria to Syrian Arab Republic
#d Convert Macedonia to Republic of Macedonia
#d Convert Great Britain and Northern Ireland to United Kingdom of Great Britain and Northern Ireland
#d Convert Central African to Central African Republic
#d Convert Venezuela to Bolivarian Republic of Venezuela
#d Convert Serbia and montenegro to Serbia
#d Convert Brunei to Nation of Brunei
#d Convert Kosovo to Republic of Kosovo
#d Convert Wallis and futuna islands to Wallis and Futuna
#** Convert Hong Kong S.A.R to Hong Kong
#d Convert Vatican City to Holy See
#d Convert Iran to Islamic Republic of Iran
#d Convert Guinea - bissau to Guinea-Bissau
#d Convert Netherlands antilles to Curacao
#d Convert Vietnam to Viet Nam
#d Convert Congo (brazzaville) to Congo-Brazzaville
#d Convert Gambia, the to Gambia
#d Convert Cape verde to Cabo Verde
#d Convert Moldova to Republic of Moldova
#** Convert St. Helena to Saint Helena, Ascension and Tristan da Cunha
#d Convert Bolivia to Plurinational State of Boliivia
#d Convert Pitcairn Islands to Pitcairn
#d Convert Tanzania to United Republic of Tanzania
#d Convert Cote d`ivoire to Cote d'ivoire
#d Convert China - taiwan to Taiwan, Province of China
import json
import requests

def return_response(country):
    
    # Convert to lowercase 
    country = country.rstrip().lower().replace('st.', 'saint')
    
    country_conversion_dict = {
        "laos"                               : "Lao",
        "macedonia"                          : "Republic of Macedonia",
        "tanzania"                           : "United Republic of Tanzania",
        "congo (brazzaville)"                : "Congo-Brazzaville",
        "congo (kinshasa)"                   : "Congo-Kinshasa",
        "moldova"                            : "Republic of Moldova",
        "cote d`ivoire"                      : "Cote d'ivoire",
        "china - taiwan"                     : "Taiwan, Province of China",
        "china-taiwan"                       : "Taiwan, Province of China",
        "zaire"                              : "Congo-Kinshasa",
        "pitcairn islands"                   : "Pitcairn",
        "cape verde"                         : "Cabo Verde",
        "gambia, the"                        : "Gambia",
        "vietnam"                            : "Viet Nam",
        "netherlands antilles"               : "Curacao",
        "guinea - bissau"                    : "Guinea-Bissau",
        "iran"                               : "Islamic Republic of Iran",
        "vatican city"                       : "Holy See",
        "saint helena"                       : "Saint Helena, Ascension and Tristan da Cunha",
        "wallis and futuna islands"          : "Wallis and Futuna",
        "kosovo"                             : "Republic of Kosovo",
        "brunei"                             : "Nation of Brunei",
        "serbia and montenegro"              : "Serbia",
        "venezuela"                          : "Bolivarian Republic of Venezuela",
        "central african"                    : "Central African Republic",
        "great britain and northern ireland" : "United Kingdom of Great Britain and Northern Ireland",
        "macau"                              : "Macao",
        "macau s.a.r."                       : "Macao",
        "syria"                              : "Syrian Arab Republic",
        "british virgin islands"             : "Virgin Islands (British)",
        "jerusalem"                          : "Israel",
        "bolivia"                            : "Bolivia (Plurinational State of)",
        "panama"                             : "Republic of Panama",
        "russia"                             : "Russian Federation",
        "the bahamas"                        : "Bahamas",
        "the gambia"                         : "Gambia"
    }
    
    try:
        cty = country_conversion_dict[country]
    except KeyError:
        
        cty = country
        
        # Handle Korea
        if 'korea' in country:
            if 'north' in country:
                cty = "Democratic People's Republic of Korea"
            else:
                cty = "Republic of Korea"
        
        # Handle Hong Kong 
        elif 'hong' in country:
            cty = 'Hong Kong'
        
        
    
    req_url = 'https://restcountries.eu/rest/v2/name/'+ cty +'?fullText=true'
    
    output = None
    
    try:
        output = json.loads(requests.get(req_url).text)[0]
    except:
        output = None
    
    #     return json.loads(requests.get(req_url).text)
    
    return output
    
#     return json.loads(requests.get(req_url).text)

In [6]:
import pandas as pd
f = pd.read_csv('FY09.txt', sep='\t')
for item in f['Country']:
    try:
        print(return_response(item)['alpha3Code'])
    except:
        print(item)

AFG
ALB
DZA
AND
AGO
ATG
ARG
ARM
AUS
AUT
AZE
BHR
BGD
BRB
BLR
BEL
BLZ
BEN
BTN
Bolivia 
BIH
BWA
BRA
BRN
BGR
BFA
MMR
BDI
KHM
CMR
CAN
CPV
CAF
TCD
CHL
China - Mainland
TWN
COL
COM
COG
COD
CRI
CIV
HRV
CUB
CYP
CZE
DNK
DJI
DMA
DOM
ECU
EGY
SLV
GNQ
ERI
EST
ETH
FSM
FJI
FIN
FRA
GAB
GEO
DEU
GHA
GBR
GRC
GRD
GTM
GIN
GNB
GUY
HTI
HND
HKG
HKG
HUN
ISL
IND
IDN
IRN
IRQ
IRL
ISR
ITA
JAM
JPN
JOR
KAZ
KEN
KIR
KOS
KWT
KGZ
LAO
LVA
LBN
LSO
LBR
LBY
LIE
LTU
LUX
Macau S.A.R. 
MKD
MDG
MWI
MYS
MDV
MLI
MLT
MRT
MUS
MEX
MDA
MCO
MNG
MNE
MAR
MOZ
NAM
NRU
NPL
NLD
NZL
NIC
NER
NGA
NOR
OMN
PAK
Palestinian Authority Travel Document 
Panama 13.1%
PNG
PRY
PRK
PER
PHL
POL
PRT
QAT
PLW
MHL
ROU
Russia 
RWA
WSM
SMR
STP
SAU
SEN
SRB
SRB
SYC
SLE
SGP
SVK
SVN
SLB
SOM
ZAF
KOR
ESP
LKA
KNA
LCA
VCT
SDN
SUR
SWZ
SWE
CHE
SYR
TJK
TZA
THA
The Bahamas 
The Gambia 
TLS
TGO
TON
TTO
TUN
TUR
TKM
TUV
UGA
UKR
ARE
Unknown Nationality Or Stateless 
URY
UZB
VUT
VAT
VEN
VNM
YEM
ZMB
ZWE


In [ ]:
f.head(20)

In [ ]:
f.head(100)